<a href="https://colab.research.google.com/github/NotsoNoob/Steel-Rod-Detection/blob/main/countandtype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install ultralytics roboflow
!pip install torch torchvision
!pip install pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 923.8/923.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [2]:
# Importing  Libraries
import os
import yaml
import torch
from ultralytics import YOLO
from roboflow import Roboflow
from google.colab import drive
import logging
logging.basicConfig(level=logging.INFO)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Mounting Google Drive (Optional but recommended for saving models)
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#  Download Dataset from Roboflow
rf = Roboflow(api_key="rQoXBg8wCY2xXag6clfA")
project = rf.workspace("jan06phase2b").project("intern_project_santhosh")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Intern_project_Santhosh-2 in yolov8:: 100%|██████████| 78/78 [00:00<00:00, 1338.47it/s]


In [5]:
# YOLO Implementation
class YOLODetector:
    def __init__(self, data_path, model_name="yolov8n.pt"):
        self.data_path = data_path
        self.model = YOLO(model_name)
        self.verify_data_structure()

    def verify_data_structure(self):
        required_dirs = ['train', 'valid', 'test']
        for dir_name in required_dirs:
            dir_path = os.path.join(self.data_path, dir_name)
            if not os.path.exists(dir_path):
                logging.warning(f"Missing directory: {dir_path}")

        yaml_path = os.path.join(self.data_path, 'data.yaml')
        if not os.path.exists(yaml_path):
            raise FileNotFoundError(f"data.yaml not found in {self.data_path}")

    def train(self, epochs=100, batch_size=16, img_size=640):
        yaml_path = os.path.join(self.data_path, 'data.yaml')

        config = {
            'epochs': epochs,
            'batch': batch_size,
            'imgsz': img_size,
            'data': yaml_path,
            'device': 'cuda' if torch.cuda.is_available() else 'cpu',
            'workers': 2,  # Reduced for Colab
            'patience': 50,
            'save': True,
            'cache': False
        }

        try:
            self.model.train(**config)
            logging.info("Training completed successfully")
        except Exception as e:
            logging.error(f"Training failed: {str(e)}")
            raise

    def predict(self, image_path, conf_threshold=0.25):
        try:
            results = self.model.predict(
                source=image_path,
                conf=conf_threshold,
                save=True
            )
            return results
        except Exception as e:
            logging.error(f"Prediction failed: {str(e)}")
            raise

In [ ]:
# Initialize and Train
# Get the dataset path from Roboflow download
dataset_path = dataset.location

# Initialize detector
detector = YOLODetector(
    data_path=dataset_path,
    model_name="yolov8n.pt"  # Using nano model for faster training
)

# Start training
detector.train(
    epochs=50,  # Reduced epochs for demonstration
    batch_size=8,  # Smaller batch size for Colab
    img_size=640
)

100%|██████████| 6.25M/6.25M [00:00<00:00, 74.4MB/s]


Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Intern_project_Santhosh-2/data.yaml, epochs=50, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=2, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 14.0MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/Intern_project_Santhosh-2/train/labels... 27 images, 0 backgrounds, 0 corrupt: 100%|██████████| 27/27 [00:00<00:00, 105.95it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      29/50         0G      1.452      1.919     0.8946        786        640: 100%|██████████| 4/4 [01:29<00:00, 22.36s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]

                   all          4         95      0.859        0.2      0.215      0.147



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#  Make Predictions
#  Predict on a test image
test_image_path = os.path.join(dataset_path, 'test/images/')
results = detector.predict(test_image_path)



NameError: name 'os' is not defined

In [ ]:
# Method 1: Using files.download() for a single file
from google.colab import files

# For a single detection image
files.download('test/images/')

# For mAP results (often in a text or CSV file)
files.download('path/to/your/mAP_results.txt')

In [ ]:
#  Save Model to Drive (Optional)
save_path = '/content/drive/MyDrive/yolo_model'
os.makedirs(save_path, exist_ok=True)
!cp runs/detect/train/weights/best.pt $save_path/

NameError: name 'os' is not defined